<a href="https://colab.research.google.com/github/K-Opoku/ml-zoomcamp-notes_assignments/blob/main/Deep_learning_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep learing Assignment

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip


--2025-12-02 02:37:40--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T03%3A34%3A50Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T02%3A34%3A36Z&ske=2025-12-02T03%3A34%3A50Z&sks=b&skv=2018-11-09&sig=eYUxmJTRgq0%2BgF%2F2WyyU%2F69FoH4YlQ9RR%2FdvuxxkSFI%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY0NDg0MSwibmJmIjoxNzY0NjQzMDQxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdG

In [ ]:
# This is there to ensure reproducibility
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader



mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# making a code to preprocess my dataset
  # Simple transforms - just resize and normalize
input_size=200
train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [15]:
from torch.utils.data import DataLoader

# Training set
train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

# Test/Validation set
val_dataset = HairDataset(
    data_dir='./data/test',   # use the provided test folder directly
    transform=val_transforms
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=20, shuffle=False)


In [16]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [17]:
# buiilding the model
import torch
import torch.nn as nn
import torch.nn.functional as F

class HairCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)     # (200→198)
        self.pool = nn.MaxPool2d(2, 2)                   # (198→99)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)                      # logits

    def forward(self, x):
        x = F.relu(self.conv1(x))    # ReLU after conv
        x = self.pool(x)             # then pool
        x = x.view(x.size(0), -1)    # flatten
        x = F.relu(self.fc1(x))      # ReLU after fc1
        x = self.fc2(x)              # raw logits for BCEWithLogitsLoss
        return x


# Trainning setup
model = HairCNN().to(device)
criterion = torch.nn.BCEWithLogitsLoss()   # expects raw logits
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)




In [18]:
!pip install torchsummary


In [19]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [20]:
# Training the model
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6877, Acc: 0.5730, Val Loss: 0.6203, Val Acc: 0.6866
Epoch 2/10, Loss: 0.6292, Acc: 0.6667, Val Loss: 0.6223, Val Acc: 0.6517
Epoch 3/10, Loss: 0.6335, Acc: 0.5930, Val Loss: 0.6109, Val Acc: 0.6368
Epoch 4/10, Loss: 0.6015, Acc: 0.6242, Val Loss: 0.6568, Val Acc: 0.6517
Epoch 5/10, Loss: 0.6060, Acc: 0.6367, Val Loss: 0.6441, Val Acc: 0.6766
Epoch 6/10, Loss: 0.5954, Acc: 0.6617, Val Loss: 0.6259, Val Acc: 0.6667
Epoch 7/10, Loss: 0.5980, Acc: 0.6579, Val Loss: 0.6273, Val Acc: 0.6617
Epoch 8/10, Loss: 0.5978, Acc: 0.6742, Val Loss: 0.5985, Val Acc: 0.6667
Epoch 9/10, Loss: 0.5631, Acc: 0.6966, Val Loss: 0.5841, Val Acc: 0.7065
Epoch 10/10, Loss: 0.5620, Acc: 0.7129, Val Loss: 0.6842, Val Acc: 0.6468


In [21]:
import numpy as np
median_acc= np.median(history['acc'])
median_acc

std_loss_train= np.std(history['loss'])
std_loss_train

np.float64(0.034625668758471215)

In [22]:
# Adding augmentation and training again
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

# Rebuild train_loader with augmented transforms
train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [23]:
# Training the model
num_epochs = 20
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(10,20):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 11/20, Loss: 0.5832, Acc: 0.6729, Val Loss: 0.5918, Val Acc: 0.6468
Epoch 12/20, Loss: 0.5490, Acc: 0.7091, Val Loss: 0.6389, Val Acc: 0.6070
Epoch 13/20, Loss: 0.5753, Acc: 0.6879, Val Loss: 0.5871, Val Acc: 0.6716
Epoch 14/20, Loss: 0.5640, Acc: 0.7041, Val Loss: 0.5826, Val Acc: 0.6766
Epoch 15/20, Loss: 0.5653, Acc: 0.7004, Val Loss: 0.5841, Val Acc: 0.6617
Epoch 16/20, Loss: 0.5501, Acc: 0.6929, Val Loss: 0.6282, Val Acc: 0.6468
Epoch 17/20, Loss: 0.5631, Acc: 0.7016, Val Loss: 0.5989, Val Acc: 0.6766
Epoch 18/20, Loss: 0.5316, Acc: 0.7029, Val Loss: 0.5994, Val Acc: 0.6816
Epoch 19/20, Loss: 0.5285, Acc: 0.7141, Val Loss: 0.6769, Val Acc: 0.6766
Epoch 20/20, Loss: 0.5209, Acc: 0.7179, Val Loss: 0.6541, Val Acc: 0.6716


In [25]:
mean_testloss= np.mean(history['val_loss'])
mean_testloss


np.float64(0.6141876165287115)

In [26]:

last5_acc = history['val_acc'][15:20]   # indices 15–19 correspond to epochs 16–20

average_acc = np.mean(last5_acc)
print("Average Test Accuracy (last 5 epochs):", average_acc)


Average Test Accuracy (last 5 epochs): nan


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
